In [1]:
# Importing Dependencies
import pandas as pd
import locale

# Data File being loaded
purchase_file = "../Resources/purchase_data.csv"

purchase_df = pd.read_csv(purchase_file)

In [2]:
# Calculate total number of players
player_count = len(purchase_df["SN"].unique())

# Creating DataFrame to display Player Count
player_num_df = pd.DataFrame({
    "Total Players": [f'{player_count}']
})
player_num_df

,Total Players
0,576


In [3]:
# Calculating the number of unique items, the average price of an item, 
# the total number of purchases from players and the total revenue

num_unique_items = len(purchase_df["Item Name"].unique())
avg_price = round(purchase_df["Price"].mean(), 2)
num_purchases = purchase_df["Purchase ID"].count()
total_revenue = purchase_df["Price"].sum()

# Creating DataFrame to display Purchasing Analysis of the Totals
purchase_analysis_totals_df = pd.DataFrame({
    "Number of Unique Items": [f'{num_unique_items}'], 
    "Average Price": [f'${avg_price}'], 
    "Number of Purchases": [f'{num_purchases}'], 
    "Total Revenue": [f'${total_revenue:,}']
})

purchase_analysis_totals_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [4]:
# Calculates count of each gender by only keeping one purchase from every unique player and creates a dataframe
unique_df = purchase_df.drop_duplicates(subset = 'SN', keep = "first")
gender_demo_df = pd.DataFrame(unique_df["Gender"].value_counts())

# Changes column title to Total Count
gender_demo_df = gender_demo_df.rename(columns={"Gender":"Total Count"})

# Calculates percentage of each gender group and adds a percentage column to dataframe
gender_rawper = gender_demo_df["Total Count"]/(player_count)
gender_percent = (round(gender_rawper * 100, 2))
gender_demo_df["Percentage of Players"] = gender_percent.astype(str) + '%'


gender_demo_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [5]:
# Creates dataframe sorted by Gender
gender_purchase_df = purchase_df.groupby(["Gender"])


# Renames first column to Purchase Count and calculates total purchases by gender
gen_pur_analysis_df = pd.DataFrame(gender_purchase_df["Purchase ID"].count())
gen_pur_analysis_df = gen_pur_analysis_df.rename(columns={"Purchase ID":"Purchase Count"})

# Calculates average price and total purchase value by gender
avg_price_by_gen = gender_purchase_df["Price"].mean()
gen_pur_analysis_df["Average Purchase Price"] = round(avg_price_by_gen, 2)
    
gen_pur_analysis_df["Total Purchase Value"] = gender_purchase_df["Price"].sum()

gen_pur_analysis_df["Avg Total Purchase per Person"] = round((gen_pur_analysis_df["Total Purchase Value"])/(gender_demo_df["Total Count"]), 2)
gen_pur_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


In [9]:
# Creates bins that will be used to sort the data by certain age demographics
bins = [0, 9, 14, 19, 24, 29, 34, 39, float("inf")]
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

purchase_df["Age Demographic"] = pd.cut(unique_df["Age"], bins, labels=group_labels)

# Divides the data by the newly created bins above
age_group = unique_df.groupby("Age Demographic")

# Creates and calculates the total count of players in each age range and their percentage out of the group overall
age_demo_df = pd.DataFrame(age_group["Age"].count())
age_demo_df = age_demo_df.rename(columns={"Age":"Total Count"})

age_percent = (age_demo_df["Total Count"])/(player_count)
age_percent = (round(age_percent * 100, 2))
age_demo_df["Percentage of Players"] = age_percent.astype(str) + '%'

age_demo_df

,Total Count,Percentage of Players
Age Demographic,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [ ]:
age_purchase_df = purchase_df.groupby(["Age Demographic"])

age_pur_analysis_df = pd.DataFrame(age_purchase_df["Purchase ID"].count())
age_pur_analysis_df = age_pur_analysis_df.rename(columns={"Purchase ID":"Purchase Count"})

avg_price_by_age = age_purchase_df["Price"].mean()
age_pur_analysis_df["Average Purchase Price"] = round(avg_price_by_age, 2)
    
age_pur_analysis_df["Total Purchase Value"] = gender_purchase_df["Price"].sum()

age_pur_analysis_df["Avg Total Purchase per Person"] = round((gen_pur_analysis_df["Total Purchase Value"])/(gender_demo_df["Total Count"]), 2)
age_pur_analysis_df